In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from lsst.sims.photUtils import Sed
from lsst.sims.photUtils import Bandpass
from lsst.sims.photUtils import PhotometricParameters

import os

In [2]:
# Let's find the full well magnitude per filter for a variety of exposure times.
filters = ['u', 'g', 'r', 'i', 'z', 'y']



In [3]:
band_dir = os.getenv("LSST_THROUGHPUTS_BASELINE")
bps = []

for filtername in filters:
    
    tempB = Bandpass()
    tempB.readThroughput(band_dir+'filter_'+filtername+'.dat')
    bps.append(tempB)

In [4]:
bps

In [5]:
wave = np.arange(bps[0].wavelen.min(), bps[-1].wavelen.max()+1)

In [6]:
const = 1e-3
flat_sed = Sed(wavelen=wave, fnu=np.zeros(wave.size, dtype=float) + const)

In [7]:
flat_sed.calcMag(bps[3])

16.400065622282231

In [8]:
params = PhotometricParameters(exptime=15, nexp=1)

In [9]:
flat_sed.calcADU(bps[0], params) 

589934.25879503158

In [74]:
params.gain  # Electrons per ADU

2.3

In [75]:
# Do I need to correct for this being a 2-d Gaussian?
def electrons_in_peak(adu, fwhm_eff, phot_params):
    # Given total number of ADUs from a star, and seeing,
    # return the 
    c = fwhm_eff/2.3548/phot_params.platescale
    peak = adu/c/np.sqrt(2.*np.pi)
    peak = peak*phot_params.gain
    return peak

In [76]:
# OK, let's make arrays of mags and peak electron counts.
electrons_in_peak(flat_sed.calcADU(bps[1], params) , 1., params)


407370.81229183014

In [ ]:
# would like to know how we lose things on the bright end.  Let's start with r

In [89]:
consts = 10.**np.arange(-5, -2, .1)

In [118]:
params = PhotometricParameters(exptime=15, nexp=1)
mags = []
peak_es = []
fwhm = 1.
bandpass = bps[1]
for con in consts:
    flat_sed = Sed(wavelen=wave, fnu=np.zeros(wave.size, dtype=float) + con)
    peak_es.append(electrons_in_peak(flat_sed.calcADU(bandpass, params), fwhm, params))
    mags.append(flat_sed.calcMag(bandpass))

In [119]:
elec_max = 90e3
mag_peak = np.interp(np.log10(elec_max), np.log10(peak_es), mags)

In [120]:
mag_peak

18.03943412122609